## One-Way ANOVA (Compare with anovacoagulation.odc from Feb21 folder)

The implementation here is not exactly anovacoagulation.odc. Instead of applying STZ or CR corner constraints and setting 
the value of one alpha to be determinate. Here we have set the value of mu0 to be 0

In [19]:
import pymc3 as pm
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import itertools

In [2]:
# Data from the odc file
ntotal=24
a=4
times = [62,60,63,59,63,67,71,64,65,66,68,66,71,67,68,68,56,62,60,61,63,64,63,59]
diets = list(map(int,list('1111')+list('222222')+list('333333')+list('44444444')))

In [3]:
# Making it a dataframe
# This is probably the more used form to store data
coag_data = pd.DataFrame({'times':times,'diets':diets})

In [61]:
# Model
with pm.Model() as model:
#    mu0 = pm.Normal('mu0',0,1000)
#    alpha0 = pm.Constant('alpha0',0)
    mu = pm.Normal('mu',0,1000,shape=len(coag_data.diets.unique()))

    tau = pm.Gamma('tau',alpha=0.001, beta=0.001)
    #pm.HalfNormal('tau',0,1000)

    sigma = pm.Deterministic('sigma',pow(1/tau,0.5))
    adiff12 = pm.Deterministic('adiff12', mu[0]-mu[1])
    adiff13 = pm.Deterministic('adiff13', mu[0]-mu[2])
    adiff14 = pm.Deterministic('adiff14', mu[0]-mu[3])
    adiff23 = pm.Deterministic('adiff23', mu[1]-mu[2])
    adiff24 = pm.Deterministic('adiff24', mu[1]-mu[3])
    adiff34 = pm.Deterministic('adiff34', mu[2]-mu[3])
    
    # not using mu0+alpha[coag_data.diets-1] because the STZ or CR constraints could not be enforced
    likelihood = pm.Normal('y',mu[coag_data.diets-1],sigma,observed=coag_data.times) 
    trace = pm.sample(3000, chains=1, tune=300)
    
#pm.traceplot(trace)
#plt.show()
pm.summary(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [tau_log__, mu]
100%|██████████| 3300/3300 [00:06<00:00, 516.97it/s]
The acceptance probability does not match the target. It is 0.970238446681791, but should be close to 0.8. Try to increase the number of tuning steps.
Only one chain was sampled, this makes it impossible to run some convergence checks


,mean,sd,mc_error,hpd_2.5,hpd_97.5
mu__0,61.025796,1.298980,0.023194,58.502431,63.530590
mu__1,65.972321,1.070916,0.022120,63.882814,68.089956
mu__2,67.993835,1.015823,0.017846,66.029052,70.022271
mu__3,60.992011,0.868966,0.020208,59.406247,62.791541
tau,0.177658,0.055959,0.001111,0.068573,0.283243
sigma,2.463779,0.404347,0.008548,1.709050,3.261137
adiff12,-4.946524,1.668231,0.031678,-8.264952,-1.735377
adiff13,-6.968038,1.628210,0.029883,-10.501191,-4.041325
adiff14,0.033785,1.555404,0.031044,-3.052639,2.922942
adiff23,-2.021514,1.444831,0.028633,-4.888604,0.789551


The value of sigma match with the result from anovacoagulation.odc. The values of mu_0, mu_1, mu_2, mu_3 should match with the values we get for the MAP estimates for mu0 + alpha_1,
mu0 + alpha_2, mu0 + alpha_3, mu0 + alpha_4 respectively from the anovacoagulation.odc.